<a href="https://colab.research.google.com/github/bhagirathtallapragada/Secure-AI-project-phase2/blob/main/Federatedlearning_100layernet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is an implementation to conduct federated learning using the CIFAR100 dataset with the following settings:
Cases you need to consider:
1) Every party has instances of every class, 2) Every party only holds instances of one class.

Reporting training and test performance in the above settings with 10, 20, 50, 100 parties. Also report how fast the training converges and time it take to train with different number of parties involved.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow_federated==0.13.1 --quiet

In [ ]:
# !pip install --upgrade tensorflow --quiet
# !pip install tensorflow

In [ ]:
# !pip uninstall --yes tensorboard tb-nightly
# !pip install --quiet --upgrade tensorflow-federated-nightly
# !pip install --quiet --upgrade nest-asyncio
# !pip install --quiet --upgrade tb-nightly

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
%load_ext tensorboard

In [ ]:
import collections
import time
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
from tensorflow.keras import utils as np_utils
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
# from keras.optimizers import Adam
from tensorflow.keras.layers import InputLayer, Reshape
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import CategoricalCrossentropy
np.random.seed(0)
import os
import random
from tqdm import tqdm
import numpy as np
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data.dataset import Dataset   
torch.backends.cudnn.benchmark=True
from torch.utils.data import DataLoader, Dataset

# Next version

In [ ]:
(x_train_100, y_train_100), (x_test_100, y_test_100) = cifar100.load_data()
x_train_100 = x_train_100.astype('float32')
x_test_100 = x_test_100.astype('float32')
x_train_100  /= 255
x_test_100 /= 255

In [ ]:
def ext_classes( data, labels,n, cat_y = True):
    data_cl=[]
    y=[]
    # print(data.shape)
    for i in range(n):
            # print(data[ np.argwhere( labels == i ).reshape(-1) ].shape)
            # print(labels.shape)
            # print(data[ np.argwhere( labels.reshape(-1) == i ).reshape(-1)].shape,data.shape)
            data_cl.append(data[ np.argwhere( labels.reshape(-1) == i ).reshape(-1) ][ : ])
            y.extend(np.full((data_cl[i].shape[0]), i, dtype=int))

    # print(np.array(data_cl).shape)
    x = np.vstack( (data_cl) )
    y = np.array(y)
    # print("In extract classes function")
    # print(x.shape,y.shape)
    return x, y

In [ ]:
#extract equal ratio of all classes for training data
x_train, y_train = ext_classes( x_train_100, y_train_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
# for the test case
x_test, y_test = ext_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
x_train = x_train.reshape(-1,3072)

In [ ]:
x_test = x_test.reshape(-1,3072)

In [ ]:
np.unique(y_test, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
 array([100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 1

## 1) Every party only holds instances of one class

#### 1) 10 Clients

In [ ]:
x_train.shape[ 0 ]

50000

In [ ]:
x_train.shape[ 0 ]//100

500

In [ ]:
# parameters
NUM_CLIENTS = 10 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
BATCH_SIZE = 5
EPOCHS = 3

data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), typ

In [ ]:
def build_model():
    model = tf.keras.models.Sequential()
    model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
    for  i in range(98):
        model.add( tf.keras.layers.Dense( 64, activation='relu' ) )
    model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

    return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )

In [ ]:
# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

epoch 0 <sparse_categorical_accuracy=0.996999979019165,loss=0.6156036257743835,keras_training_time_client_sum_sec=0.0>
epoch 1 <sparse_categorical_accuracy=0.9936666488647461,loss=0.12036780267953873,keras_training_time_client_sum_sec=0.0>
epoch 2 <sparse_categorical_accuracy=0.996666669845581,loss=0.021555190905928612,keras_training_time_client_sum_sec=0.0>
Training time: 169.59079718589783s


### Evaluation

In [ ]:
NUM_CLIENTS = 10 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ].reshape(100,1,3072)
  print( x.shape )
  y = y_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.10000000149011612,loss=3.0370593070983887,keras_training_time_client_sum_sec=0.0>


### 2) 20 Clients

In [ ]:
#extract equal ratio of all the classes for train
x_train, y_train = ext_classes( x_train_100, y_train_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
#extract equal ratio of all the classes for test
x_test, y_test = ext_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
x_test = x_test.reshape(-1,3072)

In [ ]:
x_train = x_train.reshape(-1,3072)

In [ ]:
# parameters
NUM_CLIENTS = 20 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
BATCH_SIZE = 5
EPOCHS = 3

# shuffle data
# idx = np.arange( TOTAL_SAMPLES )
# np.random.shuffle( idx )
# x_train = x_train[ idx ]
# y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), typ

In [ ]:
# define a function that builds our model
def build_model():
    model = tf.keras.models.Sequential()
    model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
    for  i in range(98):
        model.add( tf.keras.layers.Dense( 64, activation='relu' ) )
    model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

    return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )

In [ ]:
# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 0 <sparse_categorical_accuracy=0.996833324432373,loss=0.5802554488182068,keras_training_time_client_sum_sec=0.0>
epoch 1 <sparse_categorical_accuracy=0.9919999837875366,loss=0.10859827697277069,keras_training_time_client_sum_sec=0.0>
epoch 2 <sparse_categorical_accuracy=0.9965000152587891,loss=0.021834366023540497,keras_training_time_client_sum_sec=0.0>
Training time: 322.7802813053131s


### Evaluation

In [ ]:
NUM_CLIENTS = 20 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ].reshape(100,1,3072)
  print( x.shape )
  y = y_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.05000000074505806,loss=4.143675804138184,keras_training_time_client_sum_sec=0.0>


### 3) 50 Clients

In [ ]:
#extract equal ratio of all the classes for train
x_train, y_train = ext_classes( x_train_100, y_train_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
#extract equal ratio of all the classes for test
x_test, y_test = ext_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
x_test = x_test.reshape(-1,3072)
x_train = x_train.reshape(-1,3072)

In [ ]:
# parameters
NUM_CLIENTS = 50 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
BATCH_SIZE = 5
EPOCHS = 3

data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), typ

In [ ]:
# define a function that builds our model
def build_model():
    model = tf.keras.models.Sequential()
    model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
    for  i in range(98):
        model.add( tf.keras.layers.Dense( 64, activation='relu' ) )
    model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

    return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )


In [ ]:
# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 0 <sparse_categorical_accuracy=0.9967333078384399,loss=0.6145344376564026,keras_training_time_client_sum_sec=0.0>
epoch 1 <sparse_categorical_accuracy=0.9870666861534119,loss=0.1530180275440216,keras_training_time_client_sum_sec=0.0>
epoch 2 <sparse_categorical_accuracy=0.9947999715805054,loss=0.03344707190990448,keras_training_time_client_sum_sec=0.0>
Training time: 810.4315114021301s


### Evaluation

In [ ]:
NUM_CLIENTS = 50 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ].reshape(100,1,3072)
  print( x.shape )
  y = y_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.019999999552965164,loss=5.130789756774902,keras_training_time_client_sum_sec=0.0>


### 3) 100 Clients

In [ ]:
#extract equal ratio of all the classes for train
x_train, y_train = ext_classes( x_train_100, y_train_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
#extract equal ratio of all the classes for test
x_test, y_test = ext_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
x_test = x_test.reshape(-1,3072)
x_train = x_train.reshape(-1,3072)

In [ ]:
# parameters
NUM_CLIENTS = 100 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
BATCH_SIZE = 5
EPOCHS = 3

data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), typ

In [ ]:
# define a function that builds our model
def build_model():
    model = tf.keras.models.Sequential()
    model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
    for  i in range(98):
        model.add( tf.keras.layers.Dense( 64, activation='relu' ) )
    model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

    return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )

In [ ]:
# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



### Evaluation

In [ ]:
NUM_CLIENTS = 100 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ].reshape(100,1,3072)
  print( x.shape )
  y = y_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)

In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

In [ ]:
state, metrics = algorithm.next( state, data )

In [ ]:
metrics

In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

## 1) Every party has instances of every class

In [ ]:
def ext_classes( data, labels,n, cat_y = True):
    data_cl=[]
    y=[]
    # print(data.shape)
    for i in range(n):
            # print(data[ np.argwhere( labels == i ).reshape(-1) ].shape)
            # print(labels.shape)
            # print(data[ np.argwhere( labels.reshape(-1) == i ).reshape(-1)].shape,data.shape)
            data_cl.append(data[ np.argwhere( labels.reshape(-1) == i ).reshape(-1) ][ : ])
            y.extend(np.full((data_cl[i].shape[0]), i, dtype=int))

    # print(np.array(data_cl).shape)
    x = np.vstack( (data_cl) )
    y = np.array(y)
    # print("In extract classes function")
    # print(x.shape,y.shape)
    return x, y

In [ ]:
#extract equal ratio of all the classes for train
x_train, y_train = ext_classes( x_train_100, y_train_100 ,n=100,cat_y=False)
session = tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
x_test = x_test.reshape(-1,3072)
x_train = x_train.reshape(-1,3072)

In [ ]:
x_new = []
y_new = []
for i in range(500):
    for j in range(x_train.shape[0]//500):
        x_new.append(x_train[j*500+i])
        y_new.append(y_train[j*500+i])

In [ ]:
x_new = np.array(x_new)
x_new.shape

In [ ]:
y_new = np.array(y_new)
y_new.shape

### 1) 10 Clients

In [ ]:
x_train = x_new.copy()
y_train = y_new.copy()

In [ ]:

# parameters
NUM_CLIENTS = 10 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // NUM_CLIENTS # number of samples per client
BATCH_SIZE = 5
EPOCHS = 3

# shuffle data
# idx = np.arange( TOTAL_SAMPLES )
# np.random.shuffle( idx )
# x_train = x_train[ idx ]
# y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

In [ ]:
# define a function that builds our model
def build_model():
    model = tf.keras.models.Sequential()
    model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
    for  i in range(98):
        model.add( tf.keras.layers.Dense( 64, activation='relu' ) )
    model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

    return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )

In [ ]:
# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")

### Evaluation

In [ ]:
NUM_CLIENTS = 10 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // NUM_CLIENTS # number of samples per client
data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ].reshape(NUM_CLIENT_SAMPLES,1,3072)
  print( x.shape )
  y = y_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)

In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

### 2) 20 Clients

In [ ]:
#extract equal ratio of all the classes for train
x_train = x_new.copy()
y_train = y_new.copy()

In [ ]:
#extract equal ratio of all the classes for test
x_test, y_test = ext_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
# parameters
NUM_CLIENTS = 20 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // NUM_CLIENTS # number of samples per client
BATCH_SIZE = 5
EPOCHS = 3

# shuffle data
# idx = np.arange( TOTAL_SAMPLES )
# np.random.shuffle( idx )
# x_train = x_train[ idx ]
# y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

In [ ]:
# define a function that builds our model
def build_model():
    model = tf.keras.models.Sequential()
    model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
    for  i in range(98):
        model.add( tf.keras.layers.Dense( 64, activation='relu' ) )
    model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

    return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )

In [ ]:
# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")

### Evaluation

In [ ]:
NUM_CLIENTS = 20 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // NUM_CLIENTS # number of samples per client
data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ].reshape(NUM_CLIENT_SAMPLES,1,3072)
  print( x.shape )
  y = y_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)

In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

### 3) 50 Clients

In [ ]:
#extract equal ratio of all the classes for train
x_train = x_new.copy()
y_train = y_new.copy()

In [ ]:
#extract equal ratio of all the classes for test
x_test, y_test = ext_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
# parameters
NUM_CLIENTS = 50 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // NUM_CLIENTS # number of samples per client
BATCH_SIZE = 5
EPOCHS = 3

# shuffle data
# idx = np.arange( TOTAL_SAMPLES )
# np.random.shuffle( idx )
# x_train = x_train[ idx ]
# y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

In [ ]:
# define a function that builds our model
def build_model():
    model = tf.keras.models.Sequential()
    model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
    for  i in range(98):
        model.add( tf.keras.layers.Dense( 64, activation='relu' ) )
    model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

    return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )

In [ ]:
# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")

### Evaluation

In [ ]:
NUM_CLIENTS = 50 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // NUM_CLIENTS # number of samples per client
data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ].reshape(NUM_CLIENT_SAMPLES,1,3072)
  print( x.shape )
  y = y_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)

In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

### 4) 100 Clients

In [ ]:
#extract equal ratio of all the classes for train
x_train = x_new.copy()
y_train = y_new.copy()

In [ ]:
#extract equal ratio of all the classes for test
x_test, y_test = ext_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
# parameters
NUM_CLIENTS = 100 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // NUM_CLIENTS # number of samples per client
BATCH_SIZE = 5
EPOCHS = 3

# shuffle data
# idx = np.arange( TOTAL_SAMPLES )
# np.random.shuffle( idx )
# x_train = x_train[ idx ]
# y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

In [ ]:
# define a function that builds our model
def build_model():
    model = tf.keras.models.Sequential()
    model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
    for  i in range(98):
        model.add( tf.keras.layers.Dense( 64, activation='relu' ) )
    model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

    return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )

In [ ]:
# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")

### Evaluation

In [ ]:
NUM_CLIENTS = 100 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // NUM_CLIENTS # number of samples per client
data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ].reshape(NUM_CLIENT_SAMPLES,1,3072)
  print( x.shape )
  y = y_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)

In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))